<a href="https://colab.research.google.com/github/Lakshmi-Adhikari-AI/LLM-HuggingFace/blob/main/ch6/Byte-Pair_Encoding_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧩 Byte-Pair Encoding tokenization
BPE started as a text compression algorithm and later became the backbone of tokenizers in models like GPT, GPT-2, RoBERTa, BART, and DeBERTa.  
This notebook walks you through BPE training, merging, and tokenization with hands-on code

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## 1️⃣ How Does BPE Work? (Theory)

Training steps:
1. Normalize and pre-tokenize corpus into words.
2. Build initial vocabulary from all characters in those words.
3. Iteratively add new tokens by merging the most frequent pair of existing tokens.
4. Repeat until vocab size is reached.

This algorithm learns efficient subword merges so rare words get split but common ones are compact.


## 2️⃣ BPE Training: An Example Corpus

We'll use a toy corpus demonstrating how the base vocabulary and merges are built.


In [ ]:
# Example toy corpus (simple sentences)
corpus=[
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens."
]

## 3️⃣ Pre-Tokenization using GPT-2 Rules

GPT-2 uses byte-level BPE and tracks spaces as special characters.
We'll pre-tokenize all words using GPT-2's tokenizer.


In [ ]:
from transformers import AutoTokenizer
from collections import defaultdict

tokenizer=AutoTokenizer.from_pretrained("gpt2")
word_freqs=defaultdict(int)

for text in corpus:
  words_with_offsets=tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words=[word for word,offset in words_with_offsets]
  for word in new_words:
    word_freqs[word]+=1
print(word_freqs) # Shows frequency dict, e.g. {'This': 3, 'Ġis': 2, ...}

## 4️⃣ Build the Base Vocabulary

Initialize the vocabulary with all unique characters found after normalization and pre-tokenization.


In [ ]:
alphabet=[]
for word in word_freqs.keys():
  for letter in word:
    if letter not in alphabet:
      alphabet.append(letter)
alphabet.sort()
print(alphabet) # e.g. [',', '.', 'C', 'F', 'H', ..., 'Ġ']

## 5️⃣ Prepare Token Splits for Training

Split each word into its character list for merge tracking.


In [ ]:
vocab=["<|endoftext|>"]+alphabet.copy()
splits={word:[c for c in word] for word in word_freqs.keys()}
print(splits)

## 6️⃣ Pair Frequencies: Which Pairs to Merge?

Define a helper function to count frequencies of each character pair in the current splits.


In [ ]:
def compute_pair_freqs(splits):
  pair_freqs=defaultdict(int)
  for word,freq in word_freqs.items():
    split=splits[word]
    if len(split)==1:
      continue
    for i in range(len(split)-1):
      pair=(split[i],split[i+1])
      pair_freqs[pair]+=freq
  return pair_freqs

pair_freqs=compute_pair_freqs(splits)
for i,key in enumerate(list(pair_freqs.keys())[:6]):
  print(f"{key}:{pair_freqs[key]}")

## 7️⃣ Merge the Most Frequent Pair

Find the pair with the max frequency, record the merge, and update splits and vocabulary.
Repeat until reaching target vocab size.


In [ ]:
def merge_pair(a,b,splits):
  for word in word_freqs:
    split=splits[word] # Assign the value here
    if len(split)==1:
      continue
    i=0
    while i<len(split)-1:
      if split[i]==a and split[i+1]==b:
        split=split[:i]+[a+b]+split[i+2 :]
      else:
        i+=1
    splits[word]=split
  return splits

# Example: First merge is ('G','t')
splits=merge_pair('Ġ', 't', splits)
print(splits["Ġtrained"])

## 8️⃣ BPE Training Loop: Build a Small Example Vocabulary

Let’s loop and learn up to e.g. vocab size 50.


In [ ]:
merges={('Ġ', 't'):'Ġt'}
vocab.append('Ġt')

vocab_size=50
while len(vocab)<vocab_size:
  pair_freqs=compute_pair_freqs(splits)
  best_pair=""
  max_freq=None
  for pair,freq in pair_freqs.items():
    if max_freq is None or max_freq<freq:
      best_pair=pair
      max_freq=freq
  splits=merge_pair(*best_pair,splits)
  merges[best_pair]=best_pair[0]+best_pair[1]
  vocab.append(best_pair[0]+best_pair[1])

print(merges) # Learned merge rules
print(vocab)  # All learned tokens


## 9️⃣ Tokenization With Learned Rules

Apply all merge rules, in order, to a new text to get its BPE tokenization.


In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

    # Try on a new sentence
    print(tokenize("This is not a token"))

# ✅ Summary

- **BPE** builds its vocabulary by greedy pairing and merging most frequent character pairs until target vocab size.
- Allows rare/out-of-vocabulary words to be split while keeping common words as single tokens.
- Real-world models use byte-level BPE (every possible byte is in vocab) for robustness.
- Implementing BPE at small scale helps you understand how and why LLM tokenizers work!